# IMPORT LIBRARY

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,SGDRegressor
sns.set_style(style='darkgrid')

%matplotlib inline

# PRE-MADE FUNCTIONS

## EVALUATORS

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
# from sklearn.metrics import mean_absolute_percentage_error

def eval1(predic,ytest):
  print("-----------------------------------------------------------------")
  print("r2_score (r2) ="+str(r2_score(ytest,predic)))
  print("-----------------------------------------------------------------")
  print(" PE mean absolute error (mae) = "+ str(mean_absolute_error(ytest,predic)))
  print("-----------------------------------------------------------------")
  print(" PE mean squared error (mse) = "+ str(mean_squared_error(ytest,predic)))
  print("-----------------------------------------------------------------")

## MODEL FUNCTIONS

In [ ]:
def sgmod(x_train,y_train,x_test):
  sg1=SGDRegressor(verbose=5,max_iter=2000)
  sg1.fit(x_train,y_train)
  pred=sg1.predict(x_test)
  return (pred,sg1)

In [ ]:
def lrmod(x_train,y_train,x_test):
  lr1=LinearRegression()
  lr1.fit(x_train,y_train)
  pred=lr1.predict(x_test)
  return (pred,lr1)

In [ ]:
def rfmod(x_train,y_train,x_test,p_grid):
  rf1=GridSearchCV(RandomForestRegressor(),param_grid=p_grid,verbose=5,n_jobs=-1,refit=True)
  rf1.fit(x_train,y_train)
  pred=rf1.predict(x_test)
  return (pred,rf1)

In [ ]:
def svmod(x_train,y_train,x_test,p_grid):
  svg=GridSearchCV(SVR(),param_grid=p_grid,verbose=5,n_jobs=-1,refit=True)
  svg.fit(x_train,y_train)
  pred=svg.predict(x_test)
  return (pred,svg)

## DATA SCALERS

In [ ]:
def transformer1(x_train,x_test):
  m1=MinMaxScaler()
  x_train=m1.fit_transform(x_train)
  x_test=m1.transform(x_test)
  return x_train,x_test

In [ ]:
def transformer2(x_train,x_test):
  m1=StandardScaler()
  x_train=m1.fit_transform(x_train)
  x_test=m1.transform(x_test)
  return x_train,x_test

# GETTING DATA

In [ ]:
data=pd.read_csv('../input/airpressure/Folds5x2_pp.csv')

In [ ]:
data

# DATA DESCRIPTION

In [ ]:
data.info()

In [ ]:
data.describe(include='all')

# NULL CHECK

In [ ]:
data.isna().any()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.isna().transpose(),cmap='viridis',xticklabels=False,cbar=False)

# EDA(Exploratory Data Analysis)

In [ ]:
plt.figure(figsize=(16,6))
sns.scatterplot(data=data,y='RH',x='AP',palette='rainbow')
plt.tight_layout()

In [ ]:
f,axes=plt.subplots(2,2,figsize=(15,7))

sns.scatterplot(y='PE',x='AT',data=data,ax=axes[0,0])
sns.scatterplot(y='PE',x='AP',data=data,ax=axes[0,1],color='green')
sns.scatterplot(y='PE',x='V',data=data,ax=axes[1,0],color='red')
sns.scatterplot(y='PE',x='RH',data=data,ax=axes[1,1])

plt.tight_layout()

In [ ]:
f,axes2=plt.subplots(1,2,figsize=(15,7))

sns.scatterplot(y='V',x='RH',data=data,ax=axes2[0],color='red')
sns.scatterplot(y='V',x='AP',data=data,ax=axes2[1])

plt.tight_layout()

In [ ]:
f,axes2=plt.subplots(1,3,figsize=(15,7))

sns.scatterplot(y='AT',x='RH',data=data,ax=axes2[0])
sns.scatterplot(y='AT',x='AP',data=data,ax=axes2[1])
sns.scatterplot(y='AT',x='V',data=data,ax=axes2[2])

plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,8))

sns.pairplot(data=data, diag_kind='kde',palette='rainbow',corner=True)

plt.tight_layout()


In [ ]:
corr = data.corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True,cmap='coolwarm')
plt.title("DATASET HEATMAP")
#plt.show()

# DATA PREPROCESS (NOT NEEDED)

# X,Y,SPLITTING

In [ ]:
y=data['PE'].values
X=data.drop(['PE'],axis=1).values

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

# DATA SCALING

In [ ]:
x_train,x_test=transformer1(x_train,x_test) #MINMAX OK   #STD NOK

# MODEL RUN

## LINEAR REGRESSION

In [ ]:
pred1,lrob=lrmod(x_train,y_train,x_test)

## RANDOM FOREST REGRESSOR (GRID SEARCH)

In [ ]:
n_est=list(range(100,700,50))
n_estimators=dict(n_estimators=n_est)
pred2,rfob=rfmod(x_train,y_train,x_test,n_estimators)

## SVR (GRID SEARCH)

In [ ]:
C=[1,10,100,1000];Ga=[1,0.1,0.01,0.001]
p=dict(C=C,gamma=Ga)
pred3,svob=svmod(x_train,y_train,x_test,p)

## STOCHASTIC GRADIENT REGRESSOR


In [ ]:
pred4,sgob=sgmod(x_train,y_train,x_test)

# EVALUATIONS

## LINEAR REGRESSION


In [ ]:
eval1(pred1,y_test)

plt.figure(figsize=(6,6))
plt.xlabel('Linear Regression predictions')
plt.ylabel('Actual Values')
sns.scatterplot(x=pred1,y=y_test)

## RANDOM FOREST

In [ ]:
eval1(pred2,y_test)

plt.figure(figsize=(6,6))
plt.xlabel('Random Forest Regression Predictions')
plt.ylabel('Actual Values')
sns.scatterplot(x=pred2,y=y_test)

print("best params=");print(rfob.best_params_)

## SVR 

In [ ]:
eval1(pred3,y_test)

plt.figure(figsize=(6,6))
plt.xlabel('SVM Predictions')
plt.ylabel('Actual Values')
sns.scatterplot(x=pred3,y=y_test)

print("best params=");print(svob.best_params_)

## STOCHASTIC GRDIENT REGRESSOR

In [ ]:
eval1(pred4,y_test)

plt.figure(figsize=(6,6))
plt.xlabel('SGD Regression predictions')
plt.ylabel('Actual Values')
sns.scatterplot(x=pred4,y=y_test)